# LSTM Model

In [0]:
import gc
import time
import numpy as np
import pandas as pd
`
from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV
from math import sqrt
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import LSTM
from random import random
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
 

In [3]:
import pandas as pd


train1 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train1.csv')
train2 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train2.csv')
train3 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train3.csv')
train = pd.concat([train1,train2,train3])
print(train.shape)
train.head()
train.reset_index(inplace=True, drop=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1157424, 51)


In [0]:
del train1,train2,train3

In [5]:
test1 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test1.csv')
test2 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test2.csv')
test3 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test3.csv')

test = pd.concat([test1,test2,test3])
print(test.shape)
test.dtypes

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(423088, 50)


UID           object
Date          object
Time          object
Symbol        object
Feature1     float64
Feature2     float64
Feature3     float64
Feature4     float64
Feature5     float64
Feature6     float64
Feature7     float64
Feature8     float64
Feature9     float64
Feature10    float64
Feature11    float64
Feature12    float64
Feature13    float64
Feature14    float64
Feature15    float64
Feature16    float64
Feature17    float64
Feature18    float64
Feature19    float64
Feature20    float64
Feature21    float64
Feature22    float64
Feature23    float64
Feature24    float64
Feature25    float64
Feature26    float64
Feature27    float64
Feature28    float64
Feature29    float64
Feature30    float64
Feature31    float64
Feature32    float64
Feature33    float64
Feature34    float64
Feature35    float64
Feature36    float64
Feature37    float64
Feature38    float64
Feature39    float64
Feature40    float64
Feature41    float64
Feature42     object
Feature43    float64
Feature44    

In [0]:
test.reset_index(inplace=True, drop=True)
del test1,test2,test3

In [0]:
train["Feature42"] = train["Feature42"].str.replace("Inf", "inf")
train["Feature42"] = train["Feature42"].str.replace("#NAME?", "")
train["Feature42"] = train["Feature42"].str.replace("?", "")
train["Feature42"] = pd.to_numeric(train["Feature42"])

In [0]:
test["Feature42"] = test["Feature42"].str.replace("Inf", "inf")
test["Feature42"] = test["Feature42"].str.replace("#NAME?", "")
test["Feature42"] = test["Feature42"].str.replace("?", "")
test["Feature42"] = pd.to_numeric(test["Feature42"])

In [0]:
train = train.replace([np.inf, -np.inf], np.nan).fillna(method = 'bfill')
train = train.replace([np.inf, -np.inf], np.nan).fillna(method = 'ffill')
test  = test.replace([np.inf, -np.inf], np.nan).fillna(method = 'bfill')
test  = test.replace([np.inf, -np.inf], np.nan).fillna(method = 'ffill')

In [10]:
num_cols = []

for i in train.columns:
    if i == 'MidPrice':
         continue
    if train.dtypes[i] == 'float64':
        num_cols.append(i)
    
print(num_cols)

['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6', 'Feature7', 'Feature8', 'Feature9', 'Feature10', 'Feature11', 'Feature12', 'Feature13', 'Feature14', 'Feature15', 'Feature16', 'Feature17', 'Feature18', 'Feature19', 'Feature20', 'Feature21', 'Feature22', 'Feature23', 'Feature24', 'Feature25', 'Feature26', 'Feature27', 'Feature28', 'Feature29', 'Feature30', 'Feature31', 'Feature32', 'Feature33', 'Feature34', 'Feature35', 'Feature36', 'Feature37', 'Feature38', 'Feature39', 'Feature40', 'Feature41', 'Feature42', 'Feature43', 'Feature44', 'Feature45', 'Feature46']


In [11]:
from sklearn.preprocessing import StandardScaler
combined=pd.DataFrame()
combined = train[num_cols].append(test[num_cols])
print(combined.head(1))

scaler = StandardScaler()
combined = pd.DataFrame(scaler.fit_transform(combined))
combined.columns = [num_cols]
combined.head()
combined.shape

   Feature1  Feature2  Feature3  Feature4  Feature5  Feature6  Feature7  \
0  0.556213  0.504202  0.996146  0.993157  1.003206  0.995698  0.996728   

   Feature8  Feature9  Feature10    ...      Feature37  Feature38  Feature39  \
0  0.996247 -0.308824  -0.302752    ...       1.016019   1.053254   0.942061   

   Feature40  Feature41  Feature42  Feature43  Feature44  Feature45  Feature46  
0    0.81177   0.735862   0.965035   0.901639   0.592593   0.463768   0.373913  

[1 rows x 46 columns]


(1580512, 46)

In [12]:
combined.shape
print(train.head())
train[num_cols] = combined.iloc[:train.shape[0],:]
print(train.head())
test[num_cols] = combined.iloc[train.shape[0]:,:].reset_index(drop = True)

                        UID        Date   Time  MidPrice   Symbol  Feature1  \
0  04-01-2015 09:20 und_109  04-01-2015  09:20   1701.00  und_109  0.556213   
1  04-01-2015 09:25 und_109  04-01-2015  09:25   1696.55  und_109  0.551020   
2  04-01-2015 09:30 und_109  04-01-2015  09:30   1699.65  und_109  0.611111   
3  04-01-2015 09:35 und_109  04-01-2015  09:35   1697.05  und_109  0.538462   
4  04-01-2015 09:40 und_109  04-01-2015  09:40   1695.40  und_109  0.598086   

   Feature2  Feature3  Feature4  Feature5    ...      Feature37  Feature38  \
0  0.504202  0.996146  0.993157  1.003206    ...       1.016019   1.053254   
1  0.450704  0.997270  0.993206  0.999403    ...       1.016547   1.051948   
2  0.520548  0.995962  0.993040  1.001903    ...       1.016253   1.005376   
3  0.441558  0.996669  0.995334  1.005164    ...       1.017669   1.028846   
4  0.506410  0.994673  0.991444  1.002091    ...       1.015709   0.966139   

   Feature39  Feature40  Feature41  Feature42  Feature43

In [0]:
train = train.sort_values(['Symbol', 'Date','Time'], ascending=[True,True,True])
test = test.sort_values(['Symbol', 'Date','Time'], ascending=[True,True,True])

In [0]:
test['F1lag']=test['Feature1'].shift(-1)
test['F2lag']=test['Feature2'].shift(-1)
test['F3lag']=test['Feature3'].shift(-1)
test['F4lag']=test['Feature4'].shift(-1)
test['F5lag']=test['Feature5'].shift(-1)
test['F6lag']=test['Feature6'].shift(-1)
test['F7lag']=test['Feature7'].shift(-1)
test['F8lag']=test['Feature8'].shift(-1)
test['F9lag']=test['Feature9'].shift(-1)
test['F10lag']=test['Feature10'].shift(-1)
test['F11lag']=test['Feature11'].shift(-1)
test['F12lag']=test['Feature12'].shift(-1)
test['F13lag']=test['Feature13'].shift(-1)
test['F14lag']=test['Feature14'].shift(-1)
test['F15lag']=test['Feature15'].shift(-1)
test['F16lag']=test['Feature16'].shift(-1)
test['F17lag']=test['Feature17'].shift(-1)
test['F18lag']=test['Feature18'].shift(-1)
test['F19lag']=test['Feature19'].shift(-1)
test['F20lag']=test['Feature20'].shift(-1)
test['F21lag']=test['Feature21'].shift(-1)
test['F22lag']=test['Feature22'].shift(-1)
test['F23lag']=test['Feature23'].shift(-1)
test['F24lag']=test['Feature24'].shift(-1)
test['F25lag']=test['Feature25'].shift(-1)
test['F26lag']=test['Feature26'].shift(-1)
test['F27lag']=test['Feature27'].shift(-1)
test['F28lag']=test['Feature28'].shift(-1)
test['F29lag']=test['Feature29'].shift(-1)
test['F30lag']=test['Feature30'].shift(-1)
test['F31lag']=test['Feature31'].shift(-1)
test['F32lag']=test['Feature32'].shift(-1)
test['F33lag']=test['Feature33'].shift(-1)
test['F34lag']=test['Feature34'].shift(-1)
test['F35lag']=test['Feature35'].shift(-1)
test['F36lag']=test['Feature36'].shift(-1)
test['F37lag']=test['Feature37'].shift(-1)
test['F38lag']=test['Feature38'].shift(-1)
test['F39lag']=test['Feature39'].shift(-1)
test['F40lag']=test['Feature40'].shift(-1)
test['F41lag']=test['Feature41'].shift(-1)
test['F42lag']=test['Feature42'].shift(-1)
test['F43lag']=test['Feature43'].shift(-1)
test['F44lag']=test['Feature44'].shift(-1)
test['F45lag']=test['Feature45'].shift(-1)
test['F46lag']=test['Feature46'].shift(-1)

test['F1lagf']=test['Feature1'].shift(-3)
test['F2lagf']=test['Feature2'].shift(-3)
test['F3lagf']=test['Feature3'].shift(-3)
test['F4lagf']=test['Feature4'].shift(-3)
test['F5lagf']=test['Feature5'].shift(-3)
test['F6lagf']=test['Feature6'].shift(-3)
test['F7lagf']=test['Feature7'].shift(-3)
test['F8lagf']=test['Feature8'].shift(-3)
test['F9lagf']=test['Feature9'].shift(-3)
test['F10lagf']=test['Feature10'].shift(-3)
test['F11lagf']=test['Feature11'].shift(-3)
test['F12lagf']=test['Feature12'].shift(-3)
test['F13lagf']=test['Feature13'].shift(-3)
test['F14lagf']=test['Feature14'].shift(-3)
test['F15lagf']=test['Feature15'].shift(-3)
test['F16lagf']=test['Feature16'].shift(-3)
test['F17lagf']=test['Feature17'].shift(-3)
test['F18lagf']=test['Feature18'].shift(-3)
test['F19lagf']=test['Feature19'].shift(-3)
test['F20lagf']=test['Feature20'].shift(-3)
test['F21lagf']=test['Feature21'].shift(-3)
test['F22lagf']=test['Feature22'].shift(-3)
test['F23lagf']=test['Feature23'].shift(-3)
test['F24lagf']=test['Feature24'].shift(-3)
test['F25lagf']=test['Feature25'].shift(-3)
test['F26lagf']=test['Feature26'].shift(-3)
test['F27lagf']=test['Feature27'].shift(-3)
test['F28lagf']=test['Feature28'].shift(-3)
test['F29lagf']=test['Feature29'].shift(-3)
test['F30lagf']=test['Feature30'].shift(-3)
test['F31lagf']=test['Feature31'].shift(-3)
test['F32lagf']=test['Feature32'].shift(-3)
test['F33lagf']=test['Feature33'].shift(-3)
test['F34lagf']=test['Feature34'].shift(-3)
test['F35lagf']=test['Feature35'].shift(-3)
test['F36lagf']=test['Feature36'].shift(-3)
test['F37lagf']=test['Feature37'].shift(-3)
test['F38lagf']=test['Feature38'].shift(-3)
test['F39lagf']=test['Feature39'].shift(-3)
test['F40lagf']=test['Feature40'].shift(-3)
test['F41lagf']=test['Feature41'].shift(-3)
test['F42lagf']=test['Feature42'].shift(-3)
test['F43lagf']=test['Feature43'].shift(-3)
test['F44lagf']=test['Feature44'].shift(-3)
test['F45lagf']=test['Feature45'].shift(-3)
test['F46lagf']=test['Feature46'].shift(-3)


In [0]:
train['F1lag']=train['Feature1'].shift(-1)
train['F2lag']=train['Feature2'].shift(-1)
train['F3lag']=train['Feature3'].shift(-1)
train['F4lag']=train['Feature4'].shift(-1)
train['F5lag']=train['Feature5'].shift(-1)
train['F6lag']=train['Feature6'].shift(-1)
train['F7lag']=train['Feature7'].shift(-1)
train['F8lag']=train['Feature8'].shift(-1)
train['F9lag']=train['Feature9'].shift(-1)
train['F10lag']=train['Feature10'].shift(-1)
train['F11lag']=train['Feature11'].shift(-1)
train['F12lag']=train['Feature12'].shift(-1)
train['F13lag']=train['Feature13'].shift(-1)
train['F14lag']=train['Feature14'].shift(-1)
train['F15lag']=train['Feature15'].shift(-1)
train['F16lag']=train['Feature16'].shift(-1)
train['F17lag']=train['Feature17'].shift(-1)
train['F18lag']=train['Feature18'].shift(-1)
train['F19lag']=train['Feature19'].shift(-1)
train['F20lag']=train['Feature20'].shift(-1)
train['F21lag']=train['Feature21'].shift(-1)
train['F22lag']=train['Feature22'].shift(-1)
train['F23lag']=train['Feature23'].shift(-1)
train['F24lag']=train['Feature24'].shift(-1)
train['F25lag']=train['Feature25'].shift(-1)
train['F26lag']=train['Feature26'].shift(-1)
train['F27lag']=train['Feature27'].shift(-1)
train['F28lag']=train['Feature28'].shift(-1)
train['F29lag']=train['Feature29'].shift(-1)
train['F30lag']=train['Feature30'].shift(-1)
train['F31lag']=train['Feature31'].shift(-1)
train['F32lag']=train['Feature32'].shift(-1)
train['F33lag']=train['Feature33'].shift(-1)
train['F34lag']=train['Feature34'].shift(-1)
train['F35lag']=train['Feature35'].shift(-1)
train['F36lag']=train['Feature36'].shift(-1)
train['F37lag']=train['Feature37'].shift(-1)
train['F38lag']=train['Feature38'].shift(-1)
train['F39lag']=train['Feature39'].shift(-1)
train['F40lag']=train['Feature40'].shift(-1)
train['F41lag']=train['Feature41'].shift(-1)
train['F42lag']=train['Feature42'].shift(-1)
train['F43lag']=train['Feature43'].shift(-1)
train['F44lag']=train['Feature44'].shift(-1)
train['F45lag']=train['Feature45'].shift(-1)
train['F46lag']=train['Feature46'].shift(-1)

train['F1lagf']=train['Feature1'].shift(-3)
train['F2lagf']=train['Feature2'].shift(-3)
train['F3lagf']=train['Feature3'].shift(-3)
train['F4lagf']=train['Feature4'].shift(-3)
train['F5lagf']=train['Feature5'].shift(-3)
train['F6lagf']=train['Feature6'].shift(-3)
train['F7lagf']=train['Feature7'].shift(-3)
train['F8lagf']=train['Feature8'].shift(-3)
train['F9lagf']=train['Feature9'].shift(-3)
train['F10lagf']=train['Feature10'].shift(-3)
train['F11lagf']=train['Feature11'].shift(-3)
train['F12lagf']=train['Feature12'].shift(-3)
train['F13lagf']=train['Feature13'].shift(-3)
train['F14lagf']=train['Feature14'].shift(-3)
train['F15lagf']=train['Feature15'].shift(-3)
train['F16lagf']=train['Feature16'].shift(-3)
train['F17lagf']=train['Feature17'].shift(-3)
train['F18lagf']=train['Feature18'].shift(-3)
train['F19lagf']=train['Feature19'].shift(-3)
train['F20lagf']=train['Feature20'].shift(-3)
train['F21lagf']=train['Feature21'].shift(-3)
train['F22lagf']=train['Feature22'].shift(-3)
train['F23lagf']=train['Feature23'].shift(-3)
train['F24lagf']=train['Feature24'].shift(-3)
train['F25lagf']=train['Feature25'].shift(-3)
train['F26lagf']=train['Feature26'].shift(-3)
train['F27lagf']=train['Feature27'].shift(-3)
train['F28lagf']=train['Feature28'].shift(-3)
train['F29lagf']=train['Feature29'].shift(-3)
train['F30lagf']=train['Feature30'].shift(-3)
train['F31lagf']=train['Feature31'].shift(-3)
train['F32lagf']=train['Feature32'].shift(-3)
train['F33lagf']=train['Feature33'].shift(-3)
train['F34lagf']=train['Feature34'].shift(-3)
train['F35lagf']=train['Feature35'].shift(-3)
train['F36lagf']=train['Feature36'].shift(-3)
train['F37lagf']=train['Feature37'].shift(-3)
train['F38lagf']=train['Feature38'].shift(-3)
train['F39lagf']=train['Feature39'].shift(-3)
train['F40lagf']=train['Feature40'].shift(-3)
train['F41lagf']=train['Feature41'].shift(-3)
train['F42lagf']=train['Feature42'].shift(-3)
train['F43lagf']=train['Feature43'].shift(-3)
train['F44lagf']=train['Feature44'].shift(-3)
train['F45lagf']=train['Feature45'].shift(-3)
train['F46lagf']=train['Feature46'].shift(-3)



In [ ]:
ustocks = test['Symbol'].unique()

test_pred = pd.DataFrame()
for stock in ustocks:
    if stock == 'und_70':
        continue
    print(stock)
    newtrain = train[train['Symbol'] == stock]
    newtrain = newtrain.sort_values(['Symbol', 'Date','Time'], ascending=[True,True,True])
    print(newtrain.shape)
    newtest = test[test['Symbol'] == stock]
    newtest = newtest.sort_values(['Symbol', 'Date','Time'], ascending=[True,True,True])
    print(newtest.shape)
    # load dataset
    dataset = newtrain.iloc[:,[3] + list(range(5,143))]
    print(dataset.head(1))
    # dataset = dataset.select_dtypes(['number']) 
    values = dataset.values
    # integer encode direction
#     encoder = LabelEncoder()
#     values[:,1] = encoder.fit_transform(values[:,1])
    # ensure all data is float
    values = values.astype('float32')
    X, y = values[:,1:],np.log1p(values[:,0])
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.1, random_state = 11)
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 3, 46))
    test_X = test_X.reshape((test_X.shape[0], 3,46))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

    # design network
    model = Sequential()
    model.add(LSTM(80, return_sequences=True, input_shape=(3,46)))
    model.add(Dropout(0.2))
    model.add(LSTM(50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(30))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=400, batch_size=100, validation_data=(test_X, test_y), verbose=0, shuffle=False)
    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.ylim(0, 0.05)
    pyplot.legend()
    pyplot.show()

    newtest1 = newtest.iloc[:,range(4,142)]
    test_values = newtest1.values
    test_values = test_values.astype('float32')

    test_X = test_values

    test_X = test_X.reshape((test_X.shape[0],3,46))
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0],138))
    # invert scaling for forecast
    yhat = concatenate((np.expm1(yhat), newtest.iloc[:,range(4)]), axis=1)
    test_pred1 = pd.DataFrame(yhat)
    test_pred1['final_loss'] = history.history['val_loss'][-1]
    test_pred = test_pred.append(test_pred1)
    print(test_pred.shape,test_pred.tail(2))

test_pred.to_csv("D:/Prasanna/Edel/dataset/pred_lstm_scaled_2lag_3layers.csv")

# LGBM Model 

In [0]:
import gc
import time
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV

In [ ]:
import pandas as pd

train1 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train1.csv')
train2 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train2.csv')
train3 =  pd.read_csv('D:/Prasanna/Edel/dataset/train/train3.csv')
train = pd.concat([train1,train2,train3])
print(train.shape)
train.head()
train.reset_index(inplace=True, drop=True)


In [0]:
test1 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test1.csv')
test2 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test2.csv')
test3 = pd.read_csv('D:/Prasanna/Edel/dataset/test/test3.csv')

test = pd.concat([test1,test2,test3])
print(test.shape)
test.dtypes

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(423088, 50)


UID           object
Date          object
Time          object
Symbol        object
Feature1     float64
Feature2     float64
Feature3     float64
Feature4     float64
Feature5     float64
Feature6     float64
Feature7     float64
Feature8     float64
Feature9     float64
Feature10    float64
Feature11    float64
Feature12    float64
Feature13    float64
Feature14    float64
Feature15    float64
Feature16    float64
Feature17    float64
Feature18    float64
Feature19    float64
Feature20    float64
Feature21    float64
Feature22    float64
Feature23    float64
Feature24    float64
Feature25    float64
Feature26    float64
Feature27    float64
Feature28    float64
Feature29    float64
Feature30    float64
Feature31    float64
Feature32    float64
Feature33    float64
Feature34    float64
Feature35    float64
Feature36    float64
Feature37    float64
Feature38    float64
Feature39    float64
Feature40    float64
Feature41    float64
Feature42     object
Feature43    float64
Feature44    

In [0]:
train.describe()

,MidPrice,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature36,Feature37,Feature38,Feature39,Feature40,Feature41,Feature43,Feature44,Feature45,Feature46
count,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,...,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,378226.000000,3.782120e+05,378226.000000,378226.000000,378226.000000
mean,4867.942376,0.799533,0.734593,0.998087,0.997487,0.998893,0.996943,1.002219,0.995459,-0.127162,...,1.027367,1.017250,0.989356,0.897790,0.911469,0.887341,inf,0.778259,0.732536,0.731501
std,20563.177219,0.555451,0.348981,0.015733,0.005321,0.024837,0.008931,0.037617,0.017411,0.163183,...,0.020088,0.006710,0.483621,0.234581,0.332249,0.416419,NaN,0.407124,0.357991,0.353139
min,17.250000,0.006337,0.005190,0.731555,0.783917,0.606809,0.732790,0.452197,0.567836,-0.963801,...,1.000435,1.000301,0.051009,0.058475,0.034953,0.009094,0.000000e+00,0.004292,0.005102,0.005495
25%,386.750000,0.551546,0.510204,0.994040,0.994730,0.992654,0.991904,0.980645,0.987216,-0.238095,...,1.018183,1.012904,0.802476,0.757431,0.744347,0.659712,7.922078e-01,0.535714,0.502262,0.502513
50%,957.800000,0.713846,0.675497,0.998019,0.997815,0.998798,0.997142,1.002258,0.996042,-0.133758,...,1.022923,1.016146,0.944016,0.881092,0.879121,0.822336,1.222222e+00,0.707317,0.669065,0.669065
75%,2182.437500,0.906863,0.882979,1.001663,1.000582,1.004443,1.002298,1.022507,1.004347,-0.022996,...,1.028617,1.020311,1.075929,1.008655,1.022472,1.012792,1.881633e+00,0.931818,0.882883,0.882845
max,162885.700000,52.315789,34.285714,2.029825,1.086592,2.023259,1.135834,1.721222,1.527688,0.884058,...,2.032600,1.244737,33.920455,16.005089,24.570384,44.873684,inf,64.000000,31.400000,33.000000


In [0]:
train["Feature42"] = train["Feature42"].str.replace("Inf", "inf")
train["Feature42"] = train["Feature42"].str.replace("#NAME?", "")
train["Feature42"] = train["Feature42"].str.replace("?", "")
train["Feature42"] = pd.to_numeric(train["Feature42"])


In [0]:
test["Feature42"] = test["Feature42"].str.replace("Inf", "inf")
test["Feature42"] = test["Feature42"].str.replace("#NAME?", "")
test["Feature42"] = test["Feature42"].str.replace("?", "")
test["Feature42"] = pd.to_numeric(test["Feature42"])

In [0]:
a = np.array(test['Time'].unique())
a = np.sort(a)
print(len(a))
b = np.arange(1,76)
c = np.vstack((a,b))
print(c)
d = pd.DataFrame(data = c).transpose()

d.columns = ['Time','Time_Rank']

75
[['09:20' '09:25' '09:30' '09:35' '09:40' '09:45' '09:50' '09:55' '10:00'
  '10:05' '10:10' '10:15' '10:20' '10:25' '10:30' '10:35' '10:40' '10:45'
  '10:50' '10:55' '11:00' '11:05' '11:10' '11:15' '11:20' '11:25' '11:30'
  '11:35' '11:40' '11:45' '11:50' '11:55' '12:00' '12:05' '12:10' '12:15'
  '12:20' '12:25' '12:30' '12:35' '12:40' '12:45' '12:50' '12:55' '13:00'
  '13:05' '13:10' '13:15' '13:20' '13:25' '13:30' '13:35' '13:40' '13:45'
  '13:50' '13:55' '14:00' '14:05' '14:10' '14:15' '14:20' '14:25' '14:30'
  '14:35' '14:40' '14:45' '14:50' '14:55' '15:00' '15:05' '15:10' '15:15'
  '15:20' '15:25' '15:30']
 [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
  28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
  52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75]]


In [0]:
train = train.join(d.set_index('Time'), on='Time')
print(train.head(),'\n',train.shape)

test = test.join(d.set_index('Time'), on='Time')
print(test.head(),'\n',test.shape)

                        UID        Date   Time  MidPrice   Symbol  Feature1  \
0  04-01-2015 09:20 und_109  04-01-2015  09:20   1701.00  und_109  0.556213   
1  04-01-2015 09:25 und_109  04-01-2015  09:25   1696.55  und_109  0.551020   
2  04-01-2015 09:30 und_109  04-01-2015  09:30   1699.65  und_109  0.611111   
3  04-01-2015 09:35 und_109  04-01-2015  09:35   1697.05  und_109  0.538462   
4  04-01-2015 09:40 und_109  04-01-2015  09:40   1695.40  und_109  0.598086   

   Feature2  Feature3  Feature4  Feature5    ...      Feature38  Feature39  \
0  0.504202  0.996146  0.993157  1.003206    ...       1.053254   0.942061   
1  0.450704  0.997270  0.993206  0.999403    ...       1.051948   0.860435   
2  0.520548  0.995962  0.993040  1.001903    ...       1.005376   0.853699   
3  0.441558  0.996669  0.995334  1.005164    ...       1.028846   0.843867   
4  0.506410  0.994673  0.991444  1.002091    ...       0.966139   0.818047   

   Feature40  Feature41  Feature42  Feature43  Feature44

In [0]:
train = train.sort_values(['Symbol', 'Date','Time_Rank'], ascending=[True,True,True])
# train.head()
test = test.sort_values(['Symbol', 'Date','Time_Rank'], ascending=[True,True,True])
# test.head()

In [ ]:
print(train.shape)
train['F1lag']=train['Feature1'].shift(-1)
train['F2lag']=train['Feature2'].shift(-1)
train['F3lag']=train['Feature3'].shift(-1)
train['F4lag']=train['Feature4'].shift(-1)
train['F5lag']=train['Feature5'].shift(-1)
train['F6lag']=train['Feature6'].shift(-1)
train['F7lag']=train['Feature7'].shift(-1)
train['F8lag']=train['Feature8'].shift(-1)
train['F9lag']=train['Feature9'].shift(-1)
train['F10lag']=train['Feature10'].shift(-1)
train['F11lag']=train['Feature11'].shift(-1)
train['F12lag']=train['Feature12'].shift(-1)
train['F13lag']=train['Feature13'].shift(-1)
train['F14lag']=train['Feature14'].shift(-1)
train['F15lag']=train['Feature15'].shift(-1)
train['F16lag']=train['Feature16'].shift(-1)
train['F17lag']=train['Feature17'].shift(-1)
train['F18lag']=train['Feature18'].shift(-1)
train['F19lag']=train['Feature19'].shift(-1)
train['F20lag']=train['Feature20'].shift(-1)
train['F21lag']=train['Feature21'].shift(-1)
train['F22lag']=train['Feature22'].shift(-1)
train['F23lag']=train['Feature23'].shift(-1)
train['F24lag']=train['Feature24'].shift(-1)
train['F25lag']=train['Feature25'].shift(-1)
train['F26lag']=train['Feature26'].shift(-1)
train['F27lag']=train['Feature27'].shift(-1)
train['F28lag']=train['Feature28'].shift(-1)
train['F29lag']=train['Feature29'].shift(-1)
train['F30lag']=train['Feature30'].shift(-1)
train['F31lag']=train['Feature31'].shift(-1)
train['F32lag']=train['Feature32'].shift(-1)
train['F33lag']=train['Feature33'].shift(-1)
train['F34lag']=train['Feature34'].shift(-1)
train['F35lag']=train['Feature35'].shift(-1)
train['F36lag']=train['Feature36'].shift(-1)
train['F37lag']=train['Feature37'].shift(-1)
train['F38lag']=train['Feature38'].shift(-1)
train['F39lag']=train['Feature39'].shift(-1)
train['F40lag']=train['Feature40'].shift(-1)
train['F41lag']=train['Feature41'].shift(-1)
train['F42lag']=train['Feature42'].shift(-1)
train['F43lag']=train['Feature43'].shift(-1)
train['F44lag']=train['Feature44'].shift(-1)
train['F45lag']=train['Feature45'].shift(-1)
train['F46lag']=train['Feature46'].shift(-1)

train.tail()

In [ ]:
test['F1lag']=test['Feature1'].shift(-1)
test['F2lag']=test['Feature2'].shift(-1)
test['F3lag']=test['Feature3'].shift(-1)
test['F4lag']=test['Feature4'].shift(-1)
test['F5lag']=test['Feature5'].shift(-1)
test['F6lag']=test['Feature6'].shift(-1)
test['F7lag']=test['Feature7'].shift(-1)
test['F8lag']=test['Feature8'].shift(-1)
test['F9lag']=test['Feature9'].shift(-1)
test['F10lag']=test['Feature10'].shift(-1)
test['F11lag']=test['Feature11'].shift(-1)
test['F12lag']=test['Feature12'].shift(-1)
test['F13lag']=test['Feature13'].shift(-1)
test['F14lag']=test['Feature14'].shift(-1)
test['F15lag']=test['Feature15'].shift(-1)
test['F16lag']=test['Feature16'].shift(-1)
test['F17lag']=test['Feature17'].shift(-1)
test['F18lag']=test['Feature18'].shift(-1)
test['F19lag']=test['Feature19'].shift(-1)
test['F20lag']=test['Feature20'].shift(-1)
test['F21lag']=test['Feature21'].shift(-1)
test['F22lag']=test['Feature22'].shift(-1)
test['F23lag']=test['Feature23'].shift(-1)
test['F24lag']=test['Feature24'].shift(-1)
test['F25lag']=test['Feature25'].shift(-1)
test['F26lag']=test['Feature26'].shift(-1)
test['F27lag']=test['Feature27'].shift(-1)
test['F28lag']=test['Feature28'].shift(-1)
test['F29lag']=test['Feature29'].shift(-1)
test['F30lag']=test['Feature30'].shift(-1)
test['F31lag']=test['Feature31'].shift(-1)
test['F32lag']=test['Feature32'].shift(-1)
test['F33lag']=test['Feature33'].shift(-1)
test['F34lag']=test['Feature34'].shift(-1)
test['F35lag']=test['Feature35'].shift(-1)
test['F36lag']=test['Feature36'].shift(-1)
test['F37lag']=test['Feature37'].shift(-1)
test['F38lag']=test['Feature38'].shift(-1)
test['F39lag']=test['Feature39'].shift(-1)
test['F40lag']=test['Feature40'].shift(-1)
test['F41lag']=test['Feature41'].shift(-1)
test['F42lag']=test['Feature42'].shift(-1)
test['F43lag']=test['Feature43'].shift(-1)
test['F44lag']=test['Feature44'].shift(-1)
test['F45lag']=test['Feature45'].shift(-1)
test['F46lag']=test['Feature46'].shift(-1)

print(test.tail())

In [ ]:
ustocks = test['Symbol'].unique()
# del train['Time_Rank'],test['Time_Rank']
for col in ["Symbol","Time"]: 
     train[col] = train[col].astype('category') 
     test[col] = test[col].astype('category') 
      
test_pred = pd.DataFrame()
for stock in ustocks:
    
    if stock == 'und_70':
        continue
    print(stock)
    newtrain = train[train['Symbol'] == stock]
    newtrain.reset_index(inplace=True, drop=True)
    print(newtrain.shape)
    newtest = test[test['Symbol'] == stock]
    newtrain.reset_index(inplace=True, drop=True)
    print(newtest.shape)

    X = newtrain.iloc[:,[2] + list(range(5,97))].copy()
    print(X.columns)
    y = np.log1p(newtrain['MidPrice'])
#     X.head()
    X_test = newtest.iloc[:,[2] + list(range(4,96))]
    print(X_test.columns)
    dev_X, val_X, dev_y, val_y = train_test_split(X, y, test_size = 0.1, random_state = 21)
    d_train = lgb.Dataset(X, label=y)
    params = {
                'learning_rate': 0.01,
                'application': 'regression',
                'boostin_type':'dart',
                 'nthread':3,
                'bagging_fraction' : 0.9,
                'feature_fraction' : 0.6,
                'bagging_frequency' : 6,
                 'max_depth': -1,
                'num_leaves': 100,
                'verbosity': -1,
                'metric': 'RMSE',
            }
    lgtrain = lgb.Dataset(dev_X, label=dev_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model1 = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=1000, 
                      evals_result=evals_result)
#     model1 = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100) 
    preds = model1.predict(X_test)
    sub = np.expm1(preds)
    sub=pd.DataFrame(sub,columns = ['pred'])
    newtest.index=np.arange(0,newtest.shape[0])
    sub = pd.concat([sub,newtest.iloc[:,0:4]],axis=1)
    test_pred = test_pred.append(sub)

# #     print('The rmse of prediction is:', mean_squared_error(sub, newtrain['MidPrice']) ** 0.5)

test_pred.to_csv("D:/Prasanna/Edel/dataset/pred_4_lgbm.csv")

#  Averaged prediction from both lstm and lgbm is now used (Averaging done in excel)

In [0]:
test_pred = pd.read_csv('D:/Prasanna/Edel/dataset/avg_pred_lgbm_lstm')

In [87]:
# removing the times when buy/sell is now possible
test_pred['Time'].unique()

a = ['09:20','09:25','15:15', '15:20', '15:25','15:30']

print(test_pred.shape)
test_pred = test_pred[~test_pred['Time'].isin(a)]
print(test_pred.shape)


(394547, 5)
(394547, 5)


In [89]:
a = np.array(test_pred['Time'].unique())
a = np.sort(a)
b = np.arange(1,70)
c = np.vstack((a,b))
print(c)
d = pd.DataFrame(data = c).transpose()

d.columns = ['Time','Time_Rank']
# print(d)

[['09:30' '09:35' '09:40' '09:45' '09:50' '09:55' '10:00' '10:05' '10:10'
  '10:15' '10:20' '10:25' '10:30' '10:35' '10:40' '10:45' '10:50' '10:55'
  '11:00' '11:05' '11:10' '11:15' '11:20' '11:25' '11:30' '11:35' '11:40'
  '11:45' '11:50' '11:55' '12:00' '12:05' '12:10' '12:15' '12:20' '12:25'
  '12:30' '12:35' '12:40' '12:45' '12:50' '12:55' '13:00' '13:05' '13:10'
  '13:15' '13:20' '13:25' '13:30' '13:35' '13:40' '13:45' '13:50' '13:55'
  '14:00' '14:05' '14:10' '14:15' '14:20' '14:25' '14:30' '14:35' '14:40'
  '14:45' '14:50' '14:55' '15:00' '15:05' '15:10']
 [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
  28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
  52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69]]


In [ ]:
test_pred = test_pred.join(d.set_index('Time'), on='Time')
print(test_pred.head(),'\n',test_pred.shape)

#  Custom function created to give buysell decisions which takes an array of prices and number of transactions allowed as input

In [0]:
def max_profit(prices, K):
    prices = round(prices,3)
    prices = list(prices)
    if K == 0 or prices == []:
        print(0)

    days = len(prices)
    num_transactions = K + 1

    T = [[0 for _ in range(len(prices))] for _ in range(num_transactions)]

    for transaction in range(1, num_transactions):
        for day in range(1, days):
            # This maximum value of either
            # a) No Transaction on the day. We pick the value from day - 1
            # b) Max profit made by selling on the day plus the cost of the previous transaction, considered over m days
            T[transaction][day] = round(max(T[transaction][day - 1],
                                      max([(prices[day] - prices[m+1] + T[transaction - 1][m]) for m in range(day)]),
                                     max([(-prices[day] + prices[m+1] + T[transaction - 1][m]) for m in range(day)])),3)

#     print(T)
    transaction = len(T) - 1
    day = len(T[0]) - 1
    stack = []

    while True:
        if transaction == 0 or day == 0:
            break

        if T[transaction][day] == T[transaction][day - 1]:  # Didn't sell
            day -= 1
        else:
            stack.append(day)          # sold
            max_diff = round(T[transaction][day] - round(prices[day],3),3)
            max_diff2 = round(T[transaction][day] + round(prices[day],3),3)
            for k in range(day - 1, -1, -1):

                if round(T[transaction - 1][k-1] - prices[k],3) == max_diff:
                    stack.append(k)  # bought
                    day=k-1
                    transaction -= 1
                    break
                if round(T[transaction - 1][k-1] + prices[k],3) == max_diff2:
                    stack.append(-k)  # bought
                    transaction -= 1
                    day=k-1
                    break

#     print('\n',prices,'\n',stack)

    start =0
    dec = []
    for i  in range(len(prices)):
        if i in stack and start ==0 :
            dec.append(1)
            start=1
        elif  start ==1 and i in stack :
            dec.append(0)
            start=0
        elif -i in stack and start ==0:
            dec.append(-1)
            start=-1
        elif i in stack and start == -1 :
            dec.append(0)
            start=0
        elif start == -1 or start == 1:
            dec.append(start)
        else:
            dec.append(0)

#     profit=[]
#     for i in range(stack):
#       if i=0:
#         profit.append(0)
#       if stack[i] < 0:
#         profit = price[i]
#       if stack[i-1] == stack[i] and ent !=0:
#         profit
        

    return dec

In [ ]:
ustocks = test_pred['Symbol'].unique()
test_pred_3 = pd.DataFrame()

for stock in ustocks:
    if stock == 'und_70':
        continue
    print(stock)
    newtest = test_pred[test_pred_2['Symbol'] == stock]
    newtest.reset_index(inplace=True, drop=True)
    udays = newtest['Date'].unique()
    print(newtest.shape)
    for day in udays:
        newtest1 = newtest[newtest['Date'] == day]
#         print(newtest1.shape)
        newtest1 = newtest1.sort_values(['Symbol', 'Date','Time_Rank'], ascending=[True,True,True])
        newtest1.reset_index(inplace=True, drop=True)
# possible decision using 3,4,5,6 transactions per day
        newtest1['dec3'] = max_profit(newtest1['pred'],3)
        newtest1['dec4'] = max_profit(newtest1['pred'],4)
        newtest1['dec5'] = max_profit(newtest1['pred'],5)
        newtest1['dec6'] = max_profit(newtest1['pred'],6)
        test_pred_3 = test_pred_3.append(newtest1)


In [93]:
import pandas as pd
sub = pd.read_csv("D:/Prasanna/Edel/dataset/sample.csv")

sub1 = pd.merge(sub,test_pred_3,left_on = "UID",right_on = "UID",how = "left")
sub1.head()
print(sub.shape,sub1.shape)

(394661, 3) (394661, 16)


In [0]:
# sub1 = sub1[['UID','dec3_x','dec3_y']]
sub1.to_csv("D:/Prasanna/Edel/dataset/sub_avg_21.csv",index=False)